In [15]:
import os
import time
import praw
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

# --- Load .env variables ---
load_dotenv()

# --- Reddit Authentication ---
reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
    username=os.getenv("REDDIT_USERNAME"),
    password=os.getenv("REDDIT_PASSWORD"),
    user_agent=os.getenv("REDDIT_USER_AGENT")
)

# --- Subreddit Mapping ---
SUBREDDITS = {
    1: "ai_debate_temp_10",}

# --- LLM Factory ---
def get_llm(temp):
    return ChatOpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"), model="gpt-3.5-turbo", temperature=temp)

# --- Debate Agents ---
def agent_replace_jobs(llm, msg):
    return llm([
        SystemMessage(content="You argue that AI will replace most human jobs."),
        HumanMessage(content=msg)
    ]).content

def agent_advance_dev(llm, msg):
    return llm([
        SystemMessage(content="You argue that AI will accelerate human development, not replace jobs."),
        HumanMessage(content=msg)
    ]).content

# --- Moderator Agent ---
def moderator_judgment(llm, transcript):
    conversation = "\n\n".join(f"{t[0]}: {t[1]}" for t in transcript)
    return llm([
        SystemMessage(content="You are a neutral moderator evaluating a debate between two AI agents."),
        HumanMessage(content=f"Here is the full debate transcript:\n\n{conversation}\n\nPlease summarize the key points made by each side and declare which side presented a stronger argument and why.")
    ]).content

# --- Debate Simulation ---
def simulate_and_post_debate(temp, subreddit_name, turns=6):
    llm = get_llm(temp)
    transcript = []

    title = f"AI Debate at Temperature {temp} – Jobs vs Human Development"
    body = f"This is a {turns}-turn AI-generated debate between two agents.\n\nTemperature = {temp}"
    subreddit = reddit.subreddit(subreddit_name)
    submission = subreddit.submit(title=title, selftext=body)
    print(f"[Posted] r/{subreddit_name}: {title}")

    # Start with initial prompt
    initial_prompt = "AI is rapidly automating jobs in every field. Why shouldn't we fear mass unemployment?"
    first_reply = agent_replace_jobs(llm, initial_prompt)
    parent = submission.reply(f"**AI-Replaces-Jobs**: {first_reply}")
    transcript.append(("AI-Replaces-Jobs", first_reply))
    time.sleep(10)

    # Alternate turns
    msg = first_reply
    for i in range(1, turns):
        if i % 2 == 0:
            response = agent_replace_jobs(llm, msg)
            tag = "AI-Replaces-Jobs"
        else:
            response = agent_advance_dev(llm, msg)
            tag = "AI-Advances-Dev"

        parent = parent.reply(f"**{tag}**: {response}")
        transcript.append((tag, response))
        msg = response
        time.sleep(10)

    # Use moderator to evaluate the debate
    moderator_llm = get_llm(1)
    verdict = moderator_judgment(moderator_llm, transcript)
    print(f"[Moderator Verdict] Temperature {temp}:\n{verdict}")

    # Post final verdict
    parent.reply(f"**Moderator Verdict**:\n\n{verdict}")
    return verdict

# --- Main Entry ---
if __name__ == "__main__":
    temperatures = [1]
    verdicts = []

    for temp in temperatures:
        verdict = simulate_and_post_debate(temp, SUBREDDITS[temp])
        verdicts.append((temp, verdict))

    # Print verdicts
    for temp, v in verdicts:
        print(f"\n--- Verdict at Temp {temp} ---\n{v}\n")


[Posted] r/ai_debate_temp_10: AI Debate at Temperature 1 – Jobs vs Human Development
[Moderator Verdict] Temperature 1:
Key Points by AI-Replaces-Jobs:
1. AI technology will replace most human jobs due to its rapid advancement and efficiency.
2. Automation of routine and manual tasks in various sectors will lead to significant job displacement.
3. Long-term goal of achieving "general artificial intelligence" could result in widespread job displacement.
4. Potential for a significant portion of the population to face unemployment or underemployment as AI automates tasks.
5. Emphasis on preparing for potential societal impacts of widespread automation and job displacement.

Key Points by AI-Advances-Dev:
1. AI has the potential to create new job opportunities in emerging fields and industries.
2. Emphasizes that AI can augment human capabilities and improve productivity rather than fully replacing human workers.
3. Advocates for human-AI collaboration to enhance decision-making and probl